<a href="https://colab.research.google.com/github/fedefliguer/bot_twitter_UnaTapaComoHoy/blob/master/version_mejorada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import random
import os
import tempfile
from werkzeug.utils import secure_filename
import numpy as np
from math import log
from abc import ABC, abstractmethod

# Classes

class NewsPaper(ABC): 
    @abstractmethod
    def get_url(self,new_day,new_month,current_year): 
        pass

class Clarin(NewsPaper): 
    def get_url(self,new_day,new_month,current_year): 
        clarin_lags = np.arange(2,74)
        lags = get_normalized_lags(clarin_lags)
        new_year = str(current_year - lags)
        return "http://tapas.clarin.com/tapa/" + new_year + "/" + new_month + "/" + new_day + "/" + new_year + new_month + new_day + "_thumb.jpg"


class Pagina_12(NewsPaper): 
    def get_url(self,new_day,new_month,current_year): 
        pagina_12_lags = np.arange(2,19)
        lags = get_normalized_lags(pagina_12_lags)
        new_year = current_year - lags
        if new_year == 2016:
          new_year = "2015"
          lags = 5
        if lags < 4:
          new_year = str(new_year)  
          url = "https://www.pagina12.com.ar/edicion-impresa/" + new_day + "-" + new_month + "-" + new_year
          req = requests.get(url, allow_redirects=True)
          soup = BeautifulSoup(req.text,  "html.parser")
          url = soup.findAll('img',{"class":"lazyload "})
          if len(url)>0:
            url = url[0]["data-src"]
          else:
            url = 'BAD URL'
        else:
          new_year = str(new_year)  
          url = "https://www.pagina12.com.ar/diario/principal/diario/index-" + new_year + "-" + new_month + "-" + new_day + ".html"
          req = requests.get(url, allow_redirects=True)
          soup = BeautifulSoup(req.text,  "html.parser")
          alt = "Tapa de la fecha " + new_day + "-" + new_month + "-" + new_year
          url = soup.findAll('img',{"alt":alt})
          if len(url)>0:
            url = url[0]["src"]
          else:
            url = 'BAD URL'
        return url

class Ole(NewsPaper): 
    def get_url(self,new_day,new_month,current_year): 
        ole_lags = np.arange(2,14)
        lags = get_normalized_lags(ole_lags)
        new_year = str(current_year - lags)
        url = "https://tapas2.ole.com.ar/tapa/" + new_year + "/" + new_month + "/" + new_day + "/" + "OLE_" + new_year + new_month + new_day + "_03.jpg"
    
# Functions

def get_newspaper(index):
    newspapers = {
        0: Clarin   ,
        1: Pagina_12    ,
        2: Ole
    }
    return newspapers[index]

def get_normalized_lags(all_lags):
    probs = [1/log(y,10) for y in all_lags]
    probs = np.square(probs)
    probs = probs/sum(probs)
    return np.random.choice(all_lags, 1, p=probs)[0] - 1
    

def search():
    # Get todays date
    today = date.today()

    # Set current year
    current_year = today.year

    # Set new day
    new_day = "{0:0=2d}".format(today.day)
    
    # Set new month
    new_month = "{0:0=2d}".format(today.month)
    
    # Some stuff
    array = np.random.choice(3, 1, p=[0.4, 0.4, 0.2])
    my_newspaper = get_newspaper(array[0])()
    url = my_newspaper.get_url(new_day,new_month,current_year)
    #diario = array[0]
    

    print(url)
    request = requests.get(url, stream=True)
    if request.status_code == 200:
        respuesta = 'La encontré :)'
    else:
        respuesta = 'No la encontré :('
    return respuesta + " - " + url

search()